In [ ]:
import pandas as pd
import scipy.io
import scipy.signal as ss
# import pywt
import glob
import matplotlib.pyplot as plt
import scaleogram as scg
from scaleogram import cws
import numpy as np
from math import *
import os
import cv2
from scipy.io import loadmat
import random

In [ ]:
print(os.getcwd())
os.chdir("..")

In [ ]:
main_data_dir = os.getcwd() + "\Data set"
print(os.listdir(main_data_dir))

In [ ]:
data_mat_files = glob.glob(main_data_dir + "\TrainingSet*\*")
label_path = main_data_dir + "\Label.csv"
label_df = pd.read_csv(label_path)

In [ ]:
label_df.drop(['Second_label', 'Third_label'], axis=1)

In [ ]:
label_df[label_df["Recording"] == "A0001"]["First_label"].values.tolist()[0]

In [ ]:
label = label_df['First_label'].tolist()

In [ ]:
def data_extract(signal_raw_list, start_point):
    signal_list = []
    signal_number_list = []
    
    for signal in signal_raw_list:
        mat = scipy.io.loadmat(signal)
        signal_list.append(mat['ECG'][0][0][2][:,start_point:])
        signal_number = signal.split('\\')[-1].split('.')[0]
        signal_number_list.append(signal_number)

    return (signal_list, signal_number_list)

signal_list, signal_number = data_extract(data_mat_files, 300)

In [ ]:
def median_filter(signal_raw_list):
    median_fil_list = []

    for i in range(len(signal_raw_list)):
        for j in range(12):
            median_fil = ss.medfilt(signal_raw_list[i][j])
            median_fil_list.append(median_fil)    
    return median_fil_list
    
median_fil_list = median_filter(signal_list)

In [ ]:
#Segmentation
def segmentation(signal_filt_list, seg_len, scale,
                sample_num_class1 = int, sample_num_class2 = int, sample_num_class3 = int, 
                sample_num_class4 = int, sample_num_class5 = int, sample_num_class6 = int,
                sample_num_class7 = int, sample_num_class8 = int, sample_num_class9 = int):
    count = 0
    lead = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']
    scg.set_default_wavelet('morl')
    px = 1/plt.rcParams['figure.dpi']
    
#     for i in range(len(signal_filt_list)):
#         sig_len = len(signal_filt_list[i])
#         seg_num = ceil(sig_len/seg_len)
#         for w in range(1,seg_num+1):
#             x = floor((sig_len-seg_len)/(seg_num-1)*(w-1))
#             segment = signal_filt_list[i][x:x+1600]
#             seg_label = label[floor(i/12)]
#             save_dic_seg = {"ECG_segment": segment}
            
#             # Create folder of each class
#             class_dir = main_data_dir + "\Segmentation_class_{}".format(seg_label)
#             if not os.path.exists(class_dir):
#                 os.mkdir(class_dir)
#             seg_path = class_dir + "\{}_lead{}_seg{}.mat" 
#             # Save segment to each class
#             scipy.io.savemat(seg_path.format(signal_number[floor(i/12)], lead[count], w), save_dic_seg)
            
#             if count == 11:
#                 count = 0
#             else:
#                 count += 1

    # Scaleogram dataset
#     sample_num_class_list = [sample_num_class1, sample_num_class2, sample_num_class3, 
#                             sample_num_class4, sample_num_class5, sample_num_class6,
#                             sample_num_class7, sample_num_class8, sample_num_class9]
    for classes in range(9):
        for leads in lead:
            sample_list = random.sample(glob.glob(main_data_dir + "\\Segmentation_class_{}\\*_lead{}_*.mat".format(classes+1, leads)), 
                                        2)
            savefig_folder = main_data_dir + "\\Data_set_scale{}".format(scale)
            for segment in sample_list:
                read = scipy.io.loadmat(segment)['ECG_segment'][0]
                fig_name = segment.split('\\')[-1].removesuffix('.mat') 

                if not os.path.exists(savefig_folder):
                    os.mkdir(savefig_folder)
                savefig_path = savefig_folder + "\\{}"
                scales = scg.periods2scales(np.arange(1, scale))
                fig, ax = cws(read, scales=scales, figsize=(654*px, 329*px), coi = False)
                plt.axis('off')
                plt.savefig(savefig_path.format(fig_name), 
                            bbox_inches='tight', 
                            pad_inches = 0)
                fig.clear()
                plt.close(fig)
                ax.clear()
                plt.cla()
                plt.close("all")

    # Make csv
    data_set_list = glob.glob(main_data_dir + "\\Data_set_scale{}\\*".format(scale))
    label_list = []
    lead_list = []
    for img_path in data_set_list:
        img_name = img_path.split('\\')[-1].removesuffix('.png')
        img_signal = img_name.split('_')[0]
        img_lead = img_name.split('_')[1][-1]
        label_list.append(label_df[label_df["Recording"] == img_signal]["First_label"].values.tolist()[0])
        lead_list.append(img_lead)
    dataset_df = pd.DataFrame({"Image": data_set_list,
                             "Class": label_list,
                             "Lead": lead_list})
    dataset_df = dataset_df.sample(frac = 1)
    dataset_df.to_csv(main_data_dir + "\\Data_set_scale{}\\dataset".format(scale))

In [ ]:
segmentation(signal_filt_list = median_fil_list, seg_len = 1600, scale = 500,
                sample_num_class1 = 10, sample_num_class2 = 10, sample_num_class3 = 10, 
                sample_num_class4 = 10, sample_num_class5 = 10, sample_num_class6 = 10,
                sample_num_class7 = 10, sample_num_class8 = 10, sample_num_class9 = 10)

In [ ]:
lists = [0,1,2]
df = pd.DataFrame({"A":lists, "B":[2,5,8], "C":[3,6,9]})
print(df)

In [ ]:
"lead1"[-2]

In [ ]:
label_df[label_df["Recording"] == "A0001"]["First_label"].values.tolist()[0]